In [1]:
# Import required packages
import matplotlib.pyplot as plt
import pandas as pd
from datacube import Datacube
from pprint import pprint
from odc.geo import resxy_

# Set some configurations for displaying tables nicely
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', None)

In [2]:
# Connect to datacube
dc = Datacube(app="Products_and_measurements")

## Product Discovery

In [3]:
# List Products
dc.list_products()

,name,description,license,default_crs,default_resolution
name,,,,,
ls5_c2l2_sr,ls5_c2l2_sr,USGS Landsat t Collection 2 Level-2 Surface Reflectance,CC-BY-4.0,None,None
ls5_c2l2_st,ls5_c2l2_st,USGS Landsat 5 Collection 2 Level-2 Surface Temperature,CC-BY-4.0,None,None
ls7_c2l2_sr,ls7_c2l2_sr,USGS Landsat 7 Collection 2 Level-2 Surface Reflectance,CC-BY-4.0,None,None
ls7_c2l2_st,ls7_c2l2_st,USGS Landsat 7 Collection 2 Level-2 Surface Temperature,CC-BY-4.0,None,None
ls8_c2l2_sr,ls8_c2l2_sr,USGS Landsat 8 Collection 2 Level-2 Surface Reflectance,CC-BY-4.0,None,None
ls8_c2l2_st,ls8_c2l2_st,USGS Landsat 8 Collection 2 Level-2 Surface Temperature,CC-BY-4.0,None,None
ls9_c2l2_sr,ls9_c2l2_sr,USGS Landsat 9 Collection 2 Level-2 Surface Reflectance,CC-BY-4.0,None,None
ls9_c2l2_st,ls9_c2l2_st,USGS Landsat 9 Collection 2 Level-2 Surface Temperature,CC-BY-4.0,None,None
s2_l2a,s2_l2a,"Sentinel-2a, Sentinel-2b and Sentinel-2c imagery, processed to Level 2A (Surface Reflectance) and converted to Cloud Optimized GeoTIFFs",None,None,None


In [4]:
# List measurements
dc.list_measurements()

name   dtype         units  nodata  \
product     measurement                                                  
ls5_c2l2_sr blue                    blue  uint16             1       0   
            green                  green  uint16             1       0   
            red                      red  uint16             1       0   
            nir08                  nir08  uint16             1       0   
            swir16                swir16  uint16             1       0   
            swir22                swir22  uint16             1       0   
            qa_pixel            qa_pixel  uint16     bit_index       1   
            qa_radsat          qa_radsat  uint16     bit_index       0   
            atmos_opacity  atmos_opacity   int16             1   -9999   
            cloud_qa            cloud_qa   uint8     bit_index       0   
ls5_c2l2_st lwir                    lwir  uint16        Kelvin       0   
            TRAD                    TRAD   int16  W/(m2.sr.μm)   -9999   
            URAD                    URAD   int16  W/(m2.sr.μm)   -9999   
            DRAD                    DRAD   int16  W/(m2.sr.μm)   -9999   
            ATRAN                  ATRAN   int16             1   -9999   
            EMIS                    EMIS   int16             1   -9999   
            EMSD                    EMSD   int16             1   -9999   
            CDIST                  CDIST   int16    Kilometers   -9999   
            qa_pixel            qa_pixel  uint16     bit_index       1   
            qa_radsat          qa_radsat  uint16     bit_index       0   
            qa                        qa   int16        Kelvin   -9999   
ls7_c2l2_sr blue                    blue  uint16             1       0   
            green                  green  uint16             1       0   
            red                      red  uint16             1       0   
            nir08                  nir08  uint16             1       0   
            swir16                swir16  uint16             1       0   
            swir22                swir22  uint16             1       0   
            qa_pixel            qa_pixel  uint16     bit_index       1   
            qa_radsat          qa_radsat  uint16     bit_index       0   
            atmos_opacity  atmos_opacity   int16             1   -9999   
            cloud_qa            cloud_qa   uint8     bit_index       0   
ls7_c2l2_st lwir                    lwir  uint16        Kelvin       0   
            TRAD                    TRAD   int16  W/(m2.sr.μm)   -9999   
            URAD                    URAD   int16  W/(m2.sr.μm)   -9999   
            DRAD                    DRAD   int16  W/(m2.sr.μm)   -9999   
            ATRAN                  ATRAN   int16             1   -9999   
            EMIS                    EMIS   int16             1   -9999   
            EMSD                    EMSD   int16             1   -9999   
            CDIST                  CDIST   int16    Kilometers   -9999   
            qa_pixel            qa_pixel  uint16     bit_index       1   
            qa_radsat          qa_radsat  uint16     bit_index       0   
            qa                        qa   int16        Kelvin   -9999   
ls8_c2l2_sr coastal              coastal  uint16             1       0   
            blue                    blue  uint16             1       0   
            green                  green  uint16             1       0   
            red                      red  uint16             1       0   
            nir08                  nir08  uint16             1       0   
            swir16                swir16  uint16             1       0   
            swir22                swir22  uint16             1       0   
            qa_pixel            qa_pixel  uint16     bit_index       1   
            qa_radsat          qa_radsat  uint16     bit_index       0   
            qa_aerosol        qa_aerosol   uint8     bit_index       1   
ls8_c2l2_st lwir11                lwir11

## Dataset Searching & Querying

### Finding Dataset

In [5]:
datasets = dc.find_datasets(product="s2_l2a", limit=1)
datasets

Querying product Product(name='s2_l2a', id_=9)


[]

We can also search for datasets within a specific spatial extent or time period. To do this, we supply a spatiotemporal query (i.e. a range of x- and y-coordinates defining the spatial area to load, and a range of times).

`dc.find_datasets()` will then return a subset of datasets that match this query:

In [6]:
datasets = dc.find_datasets(
    product="s2_l2a",
    x=(114, 116),
    y=(-7, -9),
    time=("2020-01-01", "2020-01-02")
)
datasets

Querying product Product(name='s2_l2a', id_=9)


[]

### Inspecting Dataset

In [7]:
datasets[0].uris

IndexError: list index out of range

In [ ]:
datasets[0].measurements

In [ ]:
datasets[0].crs

In [ ]:
datasets[0].transform

In [ ]:
# attributes and methods that are available
ds0 = datasets[0]
dir(ds0.metadata)

In [ ]:
ds0.metadata.cloud_cover

In [ ]:
getattr(ds0.metadata, 'cloud_cover')

In [ ]:
ds0.metadata.id

In [ ]:
ds0.metadata.lat

In [ ]:
ds0.metadata.lat.begin

In [ ]:
# pprint(vars(ds0))

## Load Data
Once you know the products or datasets that you are interested in, you can load data using `dc.load()`.

In [ ]:
datasets_2 = dc.find_datasets(
    product="sentinel_2_l2a",
    x=(114, 115),
    y=(-7, -8),
    time=("2020-01-01", "2020-01-02")
)
datasets_2

In [ ]:
from pyproj import CRS
crs = CRS("EPSG:9468")
print("Projected:", crs.is_projected, "Geographic:", crs.is_geographic)
print(crs)

In [ ]:
pprint(datasets_2[0].measurements)

In [ ]:
print("Available measurements:")
for meas in datasets_2[0].measurements.keys():
    print(meas)

In [ ]:
ds2 = dc.load(
    datasets=datasets_2,
    measurements= ["red", "blue", "green"],
    output_crs="EPSG:4326", #because EPSG:9468 doesn't work here
    resolution=resxy_(-0.01, 0.01)
)

In [ ]:
ds2

We can see that `dc.load` has returned an `xarray.Dataset` containing data from our two input datasets. 

> This `xarray.Dataset` includes:  
> **Dimensions**  
> This header identifies the number of timesteps returned (time: 2) as well as the number of resulting pixels in the `x` and `y` directions.
> 
> **Coordinates**  
> - time identifies the time attributed to each returned timestep.
> - x and y provide coordinates for each pixel within the returned data.  
> - spatial_ref provides information about the spatial grid used to load the data
> 
>**Data variables**  
> These are the measurements available for the loaded product.
> For every timestep (time) returned by the query, the measured value at each pixel (y, x) is returned as an array for each measurement.
> Each data variable is itself an `xarray.DataArray` object.
> 
> **Attributes**  
> Other important metadata or attributes for the loaded data

We can also inspect our loaded data by plotting it:

In [ ]:
# Plot the data (using the correct variable name)
ds2.isel(time=1)[["red", "green", "blue"]].to_array().plot.imshow(robust=True)